In [1]:
!pip install transformers

In [4]:
pip install transformers torch


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline


In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [11]:
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="eng_Latn", tgt_lang='npi_Deva', max_length = 400)

Device set to use cuda:0


In [12]:

text="""Here’s a list of 10 movies, their casts, and brief story summaries:

1. The Shawshank Redemption (1994)
Cast:

Tim Robbins as Andy Dufresne
Morgan Freeman as Ellis "Red" Redding
Bob Gunton as Warden Norton Summary: A banker, Andy Dufresne, is sentenced to life in Shawshank Prison for a crime he didn't commit. He befriends fellow inmate Red and uses his financial skills to help the prison administration, ultimately plotting his escape."""

In [13]:
translator(text)

[{'translation_text': 'यहाँ १० चलचित्रहरूको सूची छ, तिनीहरूको कास्ट, र छोटो कथा सारांशहरूः १. द श्याभ्सक रिडेम्पशन (१९९४ cast) कलाकारः टिम रोबिन्स एन्डी डफ्रेन्स मोर्गन फ्रीम्यानको रूपमा एलिस "रेड" रेडिंग बोब गन्टनको रूपमा वार्डन नर्टन सारांशः एक बैंकर, एन्डी डफ्रेन्स, एक अपराधको लागि श्याभ्सक जेलमा जीवनको लागि सजाय भोग्छन्। उनी साथी कैदी रेडसँग मित्र हुन्छन् र आफ्नो वित्तीय सीपलाई जेल प्रशासनलाई मद्दत गर्न प्रयोग गर्छन्, अन्ततः आफ्नो भाग्ने योजना बनाउँछन्।'}]

In [14]:
# Split the text by lines or sections to preserve the structure
sections = text.split('\n')

sections

['Here’s a list of 10 movies, their casts, and brief story summaries:',
 '',
 '1. The Shawshank Redemption (1994)',
 'Cast:',
 '',
 'Tim Robbins as Andy Dufresne',
 'Morgan Freeman as Ellis "Red" Redding',
 "Bob Gunton as Warden Norton Summary: A banker, Andy Dufresne, is sentenced to life in Shawshank Prison for a crime he didn't commit. He befriends fellow inmate Red and uses his financial skills to help the prison administration, ultimately plotting his escape."]

In [15]:
# Translate each section
translated_sections = [translator(section)[0]['translation_text'] for section in sections]

# Reassemble the translated text
translated_text = '\n'.join(translated_sections)

print(translated_text)

यहाँ १० वटा चलचित्रहरूको सूची छ, तिनीहरूको कास्ट, र छोटो कथा सारांशः
(ख)
१. श्वाशशंक रिडेम्पशन (१९९४)
कलाकार:
(ख)
टिम रोबिन्स एन्डी डफ्रेसनको भूमिकामा
मोर्गन फ्रीमन एलिस "रेड" रेडिंगको रूपमा
बोब गुन्टन वार्डन नर्टनको रूपमा सारांशः एक बैंकर, एन्डी डफ्रेसन, एक अपराधको लागि श्वाशान्क जेलमा जीवनको लागि सजाय भोग्छन् जुन उनले गरेनन्। उनी साथी कैदी रेडसँग मित्रता गर्छन् र आफ्नो वित्तीय सीपलाई जेल प्रशासनलाई मद्दत गर्न प्रयोग गर्छन्, अन्ततः आफ्नो भाग्ने योजना बनाउँछन्।


In [16]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="eng_Latn", tgt_lang='npi_Deva', max_length=400)

# Function to mask and unmask email addresses (you can add more patterns like URLs, phone numbers, etc.)
def mask_sensitive_content(text):
    # Find email addresses and replace them with placeholders
    emails = re.findall(r'\S+@\S+', text)
    for i, email in enumerate(emails):
        text = text.replace(email, f"[EMAIL_{i}]")
    return text, emails

def unmask_sensitive_content(text, emails):
    # Restore the original emails
    for i, email in enumerate(emails):
        text = text.replace(f"[EMAIL_{i}]", email)
    return text

text = """Here’s a list of 10 movies, their casts, and brief story summaries:

1. The Shawshank Redemption (1994)
Cast:

Tim Robbins as Andy Dufresne
Morgan Freeman as Ellis "Red" Redding
Bob Gunton as Warden Norton
Summary: A banker, Andy Dufresne, is sentenced to life in Shawshank Prison for a crime he didn't commit. He befriends fellow inmate Red and uses his financial skills to help the prison administration, ultimately plotting his escape.

For more information, you can email john.doe@example.com or visit our website."""

masked_text, emails = mask_sensitive_content(text)

# Split the masked text by lines or sections to preserve the structure
sections = masked_text.split('\n')

# Translate each section
translated_sections = [translator(section)[0]['translation_text'] for section in sections]

# Reassemble the translated text
translated_text = '\n'.join(translated_sections)

final_translated_text = unmask_sensitive_content(translated_text, emails)

print(final_translated_text)


Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


यहाँ १० वटा चलचित्रहरूको सूची छ, तिनीहरूको कास्ट, र छोटो कथा सारांशः
(ख)
१. श्वाशशंक रिडेम्पशन (१९९४)
कलाकार:
(ख)
टिम रोबिन्स एन्डी डफ्रेसनको भूमिकामा
मोर्गन फ्रीमन एलिस "रेड" रेडिंगको रूपमा
बोब गुन्टन वार्डन नर्टनको भूमिकामा
सारांशः एक बैंकर, एन्डी डफ्रेसन, एक अपराधको लागि श्वाशन्क जेलमा जीवनकै सजा सुनाइन्छ। उनी साथी कैदी रेडसँग मित्रता गर्छन् र आफ्नो वित्तीय क्षमता जेल प्रशासनलाई मद्दत गर्न प्रयोग गर्छन्, अन्ततः आफ्नो भाग्ने योजना बनाउँछन्।
(ख)
थप जानकारीको लागि, तपाईं john.doe@example.com इमेल गर्न सक्नुहुन्छ वा हाम्रो वेबसाइटमा जानुहोस्।


In [17]:
# Function to mask and unmask email addresses (you can add more patterns like URLs, phone numbers, etc.)
def mask_sensitive_content(text):
    # Find email addresses and replace them with placeholders
    emails = re.findall(r'\S+@\S+', text)
    for i, email in enumerate(emails):
        text = text.replace(email, f"[EMAIL_{i}]")
    return text, emails

def unmask_sensitive_content(text, emails):
    # Restore the original emails
    for i, email in enumerate(emails):
        text = text.replace(f"[EMAIL_{i}]", email)
    return text


In [20]:
text = """Here’s a list of 10 movies, their casts, and brief story summaries:

1. The Shawshank Redemption (1994)
Cast:

Tim Robbins as Andy Dufresne
Morgan Freeman as Ellis "Red" Redding
Bob Gunton as Warden Norton
Summary: A banker, Andy Dufresne, is sentenced to life in Shawshank Prison for a crime he didn't commit. He befriends fellow inmate Red and uses his financial skills to help the prison administration, ultimately plotting his escape.

For more information, you can email john.doe@example.com or visit our website."""

text

'Here’s a list of 10 movies, their casts, and brief story summaries:\n\n1. The Shawshank Redemption (1994)\nCast:\n\nTim Robbins as Andy Dufresne\nMorgan Freeman as Ellis "Red" Redding\nBob Gunton as Warden Norton\nSummary: A banker, Andy Dufresne, is sentenced to life in Shawshank Prison for a crime he didn\'t commit. He befriends fellow inmate Red and uses his financial skills to help the prison administration, ultimately plotting his escape.\n\nFor more information, you can email john.doe@example.com or visit our website.'

In [18]:
# Mask sensitive content
masked_text, emails = mask_sensitive_content(text)

# Split the masked text by lines or sections to preserve the structure
sections = masked_text.split('\n')

translated_sections = [translator(section)[0]['translation_text'] for section in sections]

# Reassemble the translated text
translated_text = '\n'.join(translated_sections)

final_translated_text = unmask_sensitive_content(translated_text, emails)

print(final_translated_text)


यहाँ १० वटा चलचित्रहरूको सूची छ, तिनीहरूको कास्ट, र छोटो कथा सारांशः
(ख)
१. श्वाशशंक रिडेम्पशन (१९९४)
कलाकार:
(ख)
टिम रोबिन्स एन्डी डफ्रेसनको भूमिकामा
मोर्गन फ्रीमन एलिस "रेड" रेडिंगको रूपमा
बोब गुन्टन वार्डन नर्टनको भूमिकामा
सारांशः एक बैंकर, एन्डी डफ्रेसन, एक अपराधको लागि श्वाशन्क जेलमा जीवनकै सजा सुनाइन्छ। उनी साथी कैदी रेडसँग मित्रता गर्छन् र आफ्नो वित्तीय क्षमता जेल प्रशासनलाई मद्दत गर्न प्रयोग गर्छन्, अन्ततः आफ्नो भाग्ने योजना बनाउँछन्।
(ख)
थप जानकारीको लागि, तपाईं john.doe@example.com इमेल गर्न सक्नुहुन्छ वा हाम्रो वेबसाइटमा जानुहोस्।
